In [1]:
print('Hello world!')

Hello world!


In [3]:
#import the tools 
import pandas as pd
import numpy as np

# for inconsistent entry cleaning
import re
import fuzzywuzzy
from fuzzywuzzy import process

In [8]:
df = pd.read_csv("datasets/kaggle/coffee_fix.csv")

In [9]:
# Step 1: Explore the data
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,agtron,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,50/73,9.0,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,54/77,9.0,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,54/70,9.0,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRoast House\nEthiopia Suke Quto...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,53/79,8.0,8.0,9.0,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,https://www.coffeereview.com/review/ethiopia-g...,\n\n\n94\n\n\nBig Creek Coffee Roasters\nEthi...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,48/70,9.0,9.0,9.0,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...


In [10]:
# Step 2 - Clean the data
df = df.drop(["slug", "all_text", "agtron","aroma", "acid", "body", "flavor", "aftertaste", "with_milk"], axis=1)

In [11]:
# output of the cleaned data
df.head()

,rating,roaster,name,location,origin,roast,est_price,review_date,desc_1,desc_2,desc_3
0,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...


In [12]:
# print all the unique values to obtain a sense of the data
print(f"Length of the dataframe: {len(df)}")
print("=============================")
print(f"Number of coffee brands: {len(df.name.unique())}")
print("=============================")
print(f"Number of unique locations: {len(df.location.unique())}")
print("=============================")
print(f"Average rating: {int(df.rating.mean())}")
print("=============================")

Length of the dataframe: 2282
Number of coffee brands: 2077
Number of unique locations: 296
Average rating: 92


In [16]:
# clean up location and origin
split_location = df["location"].str.split(",|;", expand=True)

#separate country adn region
df["loc_country"] = split_location[1]
df["loc_region"] = split_location[0]

# do the same for the origin column
split_origin = df["origin"].str.split(",|;", expand=True)
df["origin_1"] = split_origin[0]
df["origin_2"] = split_origin[1]
df.drop(["location", "origin"], axis=1)

def fix_text(df, col):
    # lowercase all letters
    df[col] = df[col].str.lower()
    #remove spaces at teh beginning and at the end of the string
    df[col] = df[col].str.strip()
    # capitalize each word
    df[col] = df[col].str.title()

fix_text(df, "loc_country")
fix_text(df,"loc_region")

In [18]:
# further clean up by finding closest location (90% match ratio)
def replace_matches_in_column(df, column, string_to_match, min_ratio = 55):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
titles_to_replace = ["Hawai'i", "California", "Washington"]

for i in titles_to_replace:
    replace_matches_in_column(df, "loc_country", i)

regions_to_replace = ["Taipei", "New Taipei City", "Sacramento", "Chia-Yi", "Tainan", "Nantou"]

for i in regions_to_replace:
    replace_matches_in_column(df, "loc_region", i)

In [20]:
# handling specific cases like Hong Kong, Taiwan - technically not part of China region
df.loc[df["loc_region"] == "Hong Kong",["loc_country"]] = "Hong Kong"
to_replace = {"United States":["Montana", "Pennsylvania","United States And Floyd", "Washington", "Virginia", "Connecticut", "California", "Wisconsin", "Kansas", "Minnesota", "Florida", "Wyoming", "Oregon", "Colorado", "Maine", "Kentucky", "New Hampshire", "New Jersey", "Massachusetts", "North Carolina","Illinois", "Texas", "Pennsylvania", "New York", "New Mexico", "Idaho", "Tennessee", "Ohio", "Michigan", "Oklahoma", "Vermont", "Georgia", "Louisiana", "Mississippi", "Alabama", "Maryland", "Nevada", "Virginia and Floyd", "Iowa", "D.C.", "Missouri", "Oregon", "Alaska", "Arizona", "California", "Virginia", "Colorado", "United States And Floyd"],
             "Hawai'i":["Kauai", "Hawai'i", "Maui", "Kauia"],
            "Mexico":["Oaxaca", "Mexico"],
            "Canada":["Ontario", "Nova Scotia", "Quebec", "Saskatchewan", "British Columbia", "Alberta"],
            "Taiwan":["Taiwan", "Taipei City", "Taichung", "New Taipei City", "Nantou", "New Taiwan"],
            "Australia":["Sydney"],
             "Peru":["Satipo Province"],
             "Malaysia":["Sabah"]}

def replace_values(dictionary, df_column):
    for key, value in dictionary.items():
        df_column.replace(value, key, regex=True, inplace=True)

replace_values(to_replace, df.loc_country)

df.loc_country.unique()

array(['Hong Kong', 'United States', 'Canada', 'Taiwan', nan, "Hawai'i",
       'Australia', 'Thailand', 'South Korea', 'England', 'China', None,
       'Japan', 'United Arab Emirates', 'Uganda', 'Chia-Yi', 'Slovakia',
       'Mexico', 'Indonesia', 'Laos', 'Belgium',
       'United States And Floyd', 'Guatemala', 'Peru', 'Honduras',
       'Kenya', 'Malaysia', 'New Taiwan'], dtype=object)

In [21]:
df.loc[df.loc_country.isna(), ["loc_country", "loc_region"]]

,loc_country,loc_region
15,NaN,Tainan
292,None,South Korea
713,None,Los Angeles
1748,None,Branford Connecticut
1964,None,Tainan


In [24]:
# handle the null values from the previous step manually
df.at[15, "loc_country"] = "Taiwan"
df.at[292, "loc_country"] = "South Korea"
df.at[713, "loc_country"] = "United States"
df.at[1748, "loc_country"] = "United States"
df.at[1964, "loc_country"] = "Taiwan"

df = df.drop("loc_region", axis=1)

KeyError: "['loc_region'] not found in axis"

In [25]:
df.loc[:,["origin_1", "origin_2"]]

,origin_1,origin_2
0,Panama,Ethiopia
1,Africa,Asia Pacific
2,Guji Zone,southern Ethiopia
3,Guji Zone,Oromia Region
4,Gedeb District,Gedeo Zone
...,...,...
2277,Lake Atitlán growing region,Guatemala
2278,Antigua growing region,Guatemala
2279,Antigua growing region,Guatemala
2280,Lake Atitlán growing region,Guatemala


In [26]:
# origin_1, origin_2 column fix
cols = ["origin_1", "origin_2"]

for i in cols:
    fix_text(df, i)
df["origin_2"].fillna(df["origin_1"].shift(), inplace=True)

In [27]:
df = df.drop(["location", "origin"], axis=1)

In [ ]:
df.head()